# Boundary 

The task is to combine current velocities from an oceanographic model without tides with tidal current.

* The oceanographic model data is vertically resolved and available in a vertical transect as a Dfs2 with daily timestep
* The tidal model is vertically integrated and available in a line transect as a Dfs1 with hourly timestep
* The spatial grid is identical

## Steps
1. Read data
2. Interpolate in time
3. Calculate $\mathbf{U}_{combined} = \mathbf{U}_{tide} + \mathbf{U}_{residual}$
4. Write to new dfs2

In [1]:
import numpy as np
import mikeio

dfs2 = mikeio.open("../tests/testdata/uv_vertical_daily.dfs2")
ds_surge = dfs2.read()
ds_surge

<mikeio.Dataset>
dims: (time:3, y:5, x:11)
time: 2021-08-14 12:00:00 - 2021-08-16 12:00:00 (3 records)
geometry: Grid2D (ny=5, nx=11)
items:
  0:  eastward_sea_water_velocity <u velocity component> (meter per sec)
  1:  northward_sea_water_velocity <v velocity component> (meter per sec)

In [2]:
dfs1 = mikeio.open("../tests/testdata/vu_tide_hourly.dfs1")
ds_tide = dfs1.read()
ds_tide

<mikeio.Dataset>
dims: (time:721, x:11)
time: 2021-08-01 00:00:00 - 2021-08-31 00:00:00 (721 records)
geometry: Grid1D (n=11, dx=0.09818)
items:
  0:  Tidal current component (geographic North) <v velocity component> (meter per sec)
  1:  Tidal current component (geographic East) <u velocity component> (meter per sec)

### 1. Interpolate in time

In [3]:
ds_surge_h = ds_surge.interp_time(ds_tide)
ds_surge_h

<mikeio.Dataset>
dims: (time:721, y:5, x:11)
time: 2021-08-01 00:00:00 - 2021-08-31 00:00:00 (721 records)
geometry: Grid2D (ny=5, nx=11)
items:
  0:  eastward_sea_water_velocity <u velocity component> (meter per sec)
  1:  northward_sea_water_velocity <v velocity component> (meter per sec)

### 2. Combine

**Note that the naming and order is inconsistent between the two data sources!**

In [4]:
u_tide = ds_tide["Tidal current component (geographic East)"].to_numpy()
u_tide = np.expand_dims(u_tide, 1)
u_tide.shape

(721, 1, 11)

In [5]:
u_surge = ds_surge_h["eastward_sea_water_velocity"].to_numpy()
u_surge.shape

(721, 5, 11)

In [6]:
u_combined = u_tide + u_surge
u_combined.shape

(721, 5, 11)

In [7]:
v_tide = ds_tide["Tidal current component (geographic North)"].to_numpy()
v_tide = np.expand_dims(v_tide, 1)
v_surge = ds_surge_h["northward_sea_water_velocity"].to_numpy()
v_combined = v_tide + v_surge

### 3. Write to dfs2

In [8]:
from mikeio import Dataset
ds_combined = Dataset(data=[u_combined, v_combined],
                      time=ds_tide.time,
                      items=ds_surge.items)
ds_combined

c:\users\jem\source\mikeio\mikeio\dataarray.py:664: UserWarning: Geometry is required for ndim >=1
  warnings.warn("Geometry is required for ndim >=1")


<mikeio.Dataset>
dims: (time:721, y:5, x:11)
time: 2021-08-01 00:00:00 - 2021-08-31 00:00:00 (721 records)
items:
  0:  eastward_sea_water_velocity <u velocity component> (meter per sec)
  1:  northward_sea_water_velocity <v velocity component> (meter per sec)

In [9]:
ds_combined = ds_combined.dropna()
ds_combined

<mikeio.Dataset>
dims: (time:49, y:5, x:11)
time: 2021-08-14 12:00:00 - 2021-08-16 12:00:00 (49 records)
items:
  0:  eastward_sea_water_velocity <u velocity component> (meter per sec)
  1:  northward_sea_water_velocity <v velocity component> (meter per sec)

In [10]:
dfsnew = mikeio.Dfs2()

dfsnew.write("uv_combined.dfs2", ds_combined)

c:\users\jem\source\mikeio\mikeio\dfs.py:286: UserWarning: No coordinate system provided
  warnings.warn("No coordinate system provided")


# Clean up


In [11]:
import os
os.remove("uv_combined.dfs2")